In [1]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using Plots
using StatsPlots
using Dates
using HDF5

In [2]:
function DMD_SVD(Y, r, Δt)
    
        X₋ = Y[:,1:end-1]
        X₊ = Y[:,2:end]
        U, Σ, V = svd(X₋)
        
        Uʳ = U[:, 1:r] #12 x 4
        Σʳ = diagm(Σ[1:r])
        Vʳ = V[:, 1:r]
        Ã = Uʳ' * X₊ * Vʳ / Σʳ
        Λ, W = eigen(Ã)
        Φ = X₊ * Vʳ / Σʳ * W
        Ω = log.(Λ)/Δt
        x₁ = X₋[:,1]
        b₁ = Φ \ x₁
        
        return Φ, Ω, b₁, Λ
    
end    

function DMD_reconstruct(Φ, Ω, b₁, Δt, steps)
    
        Yᵈᵐᵈ = hcat([real.(Φ * (b₁ .* exp.(Ω * (i * Δt)))) for i in 0:steps]...)
        
        return Yᵈᵐᵈ
    
end


function DMDvsERA_lme_basis(γᵢ, n)
        data_dir = "../DATA/"
    
        γᵢ = string(γᵢ)
    
        if parse(Float64, γᵢ) < 0.079 # γᵢ == "0.0" 
                ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", "0.079477")
                ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", "0.079477")
                ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", "0.079477")
                ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", "0.079477")
        else 
                ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
                ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
                ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
                ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)
        end

        
        lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
        lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series
    
        tᵉᶠ = convert.(Float64, tᵉ); tᵍᶠ = convert.(Float64, tᵍ); 
        tˣᶠ = convert.(Float64, tˣ); tʸᶠ = convert.(Float64, tʸ); 

        Hᴸᴹᴱ = [ 25.126        0
                 0             0    ]
        
        γᶠ = parse(Float64, γᵢ)

        Aᴸᴹᴱ = [ 0    √(γᶠ)
                 0     0. + 0im  ]

        tᵉᶠ = convert.(Float64, tᵉ); tᵍᶠ = convert.(Float64, tᵍ); 
        tˣᶠ = convert.(Float64, tˣ); tʸᶠ = convert.(Float64, tʸ); 

        ρᵉᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρᵉ[1], tᵉᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
        ρᵍᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρᵍ[1], tᵍᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
        ρˣᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρˣ[1], tˣᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
        ρʸᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρʸ[1], tʸᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])

        bᵉᴸᴹᴱ = LiPoSID.bloch(ρᵉᴸᴹᴱ[1:lᵐᵃˣ])
        bᵍᴸᴹᴱ = LiPoSID.bloch(ρᵍᴸᴹᴱ[1:lᵐᵃˣ])
        bˣᴸᴹᴱ = LiPoSID.bloch(ρˣᴸᴹᴱ[1:lᵐᵃˣ])
        bʸᴸᴹᴱ = LiPoSID.bloch(ρʸᴸᴹᴱ[1:lᵐᵃˣ])
    
        Y = [bᵉᴸᴹᴱ; bᵍᴸᴹᴱ; bˣᴸᴹᴱ; bʸᴸᴹᴱ]
    
        t = convert.(Float64, tᵉᶠ[1:lᵐᵃˣ])
        Δt = t[2]-t[1]

        # DMD (Dynamic mode decomposition)
        
        Φ, Ω, b₁, Λᴰᴹᴰ = DMD_SVD(Y, n, Δt)
        Yᴰᴹᴰ = DMD_reconstruct(Φ, Ω, b₁, Δt, length(t))
    
        bᵉᴰᴹᴰ = Yᴰᴹᴰ[1:3,:]
        bᵍᴰᴹᴰ = Yᴰᴹᴰ[4:6,:]
        bˣᴰᴹᴰ = Yᴰᴹᴰ[7:9,:]
        bʸᴰᴹᴰ = Yᴰᴹᴰ[10:12,:]
    
        ρᵉᴰᴹᴰ = LiPoSID.rho_series_from_bloch(bᵉᴰᴹᴰ)
        ρᵍᴰᴹᴰ = LiPoSID.rho_series_from_bloch(bᵍᴰᴹᴰ)
        ρˣᴰᴹᴰ = LiPoSID.rho_series_from_bloch(bˣᴰᴹᴰ)
        ρʸᴰᴹᴰ = LiPoSID.rho_series_from_bloch(bʸᴰᴹᴰ)
    
        ρᴰᴹᴰ = [ρᵉᴰᴹᴰ, ρᵍᴰᴹᴰ, ρˣᴰᴹᴰ, ρʸᴰᴹᴰ]
    
        # ERA (Eigenvalue Realization Algorithm)
    
        Aᴱᴿᴬ, Cᴱᴿᴬ, x₀ᴱᴿᴬ, Σᴱᴿᴬ = LiPoSID.lsid_n_ACx0Σ(Y, Δt, n) 
    
        Yᴱᴿᴬ = LiPoSID.propagate_LTI(Aᴱᴿᴬ, Cᴱᴿᴬ, x₀ᴱᴿᴬ, n, length(t))
    
        Λᴱᴿᴬ, Wᴱᴿᴬ = eigen(Aᴱᴿᴬ)
    
        bᵉᴱᴿᴬ = Yᴱᴿᴬ[1:3,:]
        bᵍᴱᴿᴬ = Yᴱᴿᴬ[4:6,:]
        bˣᴱᴿᴬ = Yᴱᴿᴬ[7:9,:]
        bʸᴱᴿᴬ = Yᴱᴿᴬ[10:12,:]
    
        ρᵉᴱᴿᴬ = LiPoSID.rho_series_from_bloch(bᵉᴱᴿᴬ)
        ρᵍᴱᴿᴬ = LiPoSID.rho_series_from_bloch(bᵍᴱᴿᴬ)
        ρˣᴱᴿᴬ = LiPoSID.rho_series_from_bloch(bˣᴱᴿᴬ)
        ρʸᴱᴿᴬ = LiPoSID.rho_series_from_bloch(bʸᴱᴿᴬ)

        ρᴱᴿᴬ = [ρᵉᴱᴿᴬ, ρᵍᴱᴿᴬ, ρˣᴱᴿᴬ, ρʸᴱᴿᴬ]
        
        return ρᴱᴿᴬ, ρᴰᴹᴰ, Λᴱᴿᴬ, Λᴰᴹᴰ, t[1:lᵐᵃˣ]
        
end

function propagate_rho_O1XY(ρ₀, ρᵉᵍˣʸ, steps)

        hcat(vec[ρᵢ] for ρᵢ in ρᵉᵍˣʸ)

        kᵉᵍˣʸ = hcat([vec(ρᵢ[1]) for ρᵢ in ρᵉᵍˣʸ]...)\vec(ρ₀)

        kᵉ, kᵍ, kˣ, kʸ = kᵉᵍˣʸ              
        ρᵉ, ρᵍ, ρˣ, ρʸ = ρᵉᵍˣʸ

        ρ = kᵉ * ρᵉ + kᵍ * ρᵍ + kˣ * ρˣ + kʸ * ρʸ

        return ρ
end 

propagate_rho_O1XY (generic function with 1 method)

In [3]:
parentdir = ".."

".."

In [4]:
data_dir = parentdir*"/DATA/"
println(data_dir)

models_dir = parentdir*"/MODELS/"
tests_dir = parentdir*"/TESTS/"

dodeca_files = ["State_D"*string(n) for n=1:20];
basis_files = ["State_B"*string(n) for n=1:4];

all_files = vcat(dodeca_files, basis_files)
test_files = dodeca_files;

../DATA/


In [5]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

tests_data_file_name = "DMDvsERA_LME_trn4_tst20_"*date_and_time_string * ".h5"

#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

γ = ["0.0", "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

println("Coupling levels to be avaluated γ ∈ ", γ)

for n in 3:5

    h5open(tests_dir*tests_data_file_name,"cw") do fid
        rank_group = create_group(fid, "rank_"*string(n)) # create rank group
    end 

    for γᵢ in  γ

        println("γᵢ = ", γᵢ)

        ρᴱᴿᴬ, ρᴰᴹᴰ, Λᴱᴿᴬ, Λᴰᴹᴰ, t = DMDvsERA_lme_basis(γᵢ, n)

        ρᵉᴰᴹᴰ, ρᵍᴰᴹᴰ, ρˣᴰᴹᴰ, ρʸᴰᴹᴰ = ρᴰᴹᴰ
        ρᵉᴱᴿᴬ, ρᵍᴱᴿᴬ, ρˣᴱᴿᴬ, ρʸᴱᴿᴬ = ρᴱᴿᴬ

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            rank_group = open_group(fid, "rank_"*string(n))
            γ_group = create_group(rank_group, "gamma_"*γᵢ) # create coupling group
            γ_group["Re_eigvals_dmd_lme"] = convert.(Float64, real.(Λᴰᴹᴰ))
            γ_group["Re_eigvals_era_lme"] = convert.(Float64, real.(Λᴱᴿᴬ))
        end 

        Hᴸᴹᴱ = [ 25.126        0
                 0             0    ]

        γᶠ = parse(Float64, γᵢ)

        Aᴸᴹᴱ = [ 0    √(γᶠ)
                 0     0. + 0im  ]


        for df_tst in test_files # loop over initial states

            print(df_tst*" ")

            if parse(Float64, γᵢ) < 0.079 # γᵢ == "0.0" 
                ρᵗˢᵗ, tᵗˢᵗ = LiPoSID.get_rho_series(data_dir*df_tst*"_2CUT_data.h5", "0.079477")

            else 
                ρᵗˢᵗ, tᵗˢᵗ = LiPoSID.get_rho_series(data_dir*df_tst*"_2CUT_data.h5", γᵢ)
            end

            if length(tᵗˢᵗ) > 1200 end_tst = 1200 else end_tst = length(tᵗˢᵗ) end

            

            tᵗˢᵗᶠ = convert.(Float64, tᵗˢᵗ)

            ρᵗˢᵗᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρᵗˢᵗ[1], tᵗˢᵗᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
            ρᵗˢᵗᴸᴹᴱ = convert(Vector{Matrix{ComplexF64}}, ρᵗˢᵗᴸᴹᴱ[1:end_tst])

            steps = min(end_tst, length(t))

            ρᵗˢᵗᴱᴿᴬ =  propagate_rho_O1XY(ρᵗˢᵗ[1], ρᴱᴿᴬ, steps)
            ρᵗˢᵗᴰᴹᴰ =  propagate_rho_O1XY(ρᵗˢᵗ[1], ρᴰᴹᴰ, steps)

            Fᴱᴿᴬ = LiPoSID.fidelity_series(basis, ρᵗˢᵗᴱᴿᴬ[1:steps], ρᵗˢᵗᴸᴹᴱ[1:steps])
            Fᴰᴹᴰ = LiPoSID.fidelity_series(basis, ρᵗˢᵗᴰᴹᴰ[1:steps], ρᵗˢᵗᴸᴹᴱ[1:steps])
            
            h5open(tests_dir*tests_data_file_name,"cw") do fid
                rank_group = open_group(fid, "rank_"*string(n)) # open rank group
                γ_group = open_group(rank_group, "gamma_"*γᵢ) # create coupling group
                state_group = create_group(γ_group, df_tst) # create coupling group

                state_group["F_dmd_lme"] = convert.(Float64, Fᴰᴹᴰ)
                state_group["F_era_lme"] = convert.(Float64, Fᴱᴿᴬ)
                state_group["time"] = convert.(Float64, tᵗˢᵗ[1:steps]) 
                
            end

        end
        
    end

end


Coupling levels to be avaluated γ ∈ 

["0.0", "0.079477", "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]


γᵢ = 0.0
State_D1 

State_D2 State_D3 State_D4 State_D5 State_D6 State_D7 

State_D8 State_D9 State_D10 State_D11 

...

State_D1 State_D2 State_D3 State_D4 State_D5 State_D6 State_D7 State_D8 

State_D9 State_D10 State_D11 State_D12 State_D13 State_D14 State_D15 State_D16 

State_D17 State_D18 State_D19 State_D20 γᵢ = 251.33
State_D1 

State_D2 State_D3 State_D4 State_D5 State_D6 State_D7 State_D8 State_D9 State_D10 State_D11 State_D12 State_D13 

State_D14 State_D15 State_D16 State_D17 State_D18 State_D19 State_D20 

In [6]:
tests_data_file_name

"DMDvsERA_LME_trn4_tst20_2023-Sep-20_at_16-51.h5"